## Import Package & Environment Setting

In [1]:
%tensorflow_version 1.x
import tensorflow as tf # version = 1.8.0
tf.__version__

TensorFlow 1.x selected.


'1.15.2'

In [2]:
import numpy as np
import pandas as pd
import time
import random
import math
import pickle
from collections import Counter, namedtuple
from collections import namedtuple
from google.colab import drive

In [3]:
# Disables AVX/FMA
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [4]:
!pip install pretty_midi
import pretty_midi

In [5]:
drive.mount('/gdrive')

Mounted at /gdrive


## Read MIDI File (TODO)

In [6]:
# song path
midi_src_path ="/gdrive/MyDrive/Classical_Music_Transformer_Integration/data/midi_source/笑忘歌_120.mid"

## Extract Chord

### Convert MIDI File to Numpy Array

In [7]:
# read midi file
data = pretty_midi.PrettyMIDI(midi_src_path)

In [8]:
# find the last point in time
max_end = -1
for instru in data.instruments:
    max_end = max(max_end, instru.notes[-1].end)
print(f'song length: {max_end}')

song length: 99.18458741458333


In [9]:
# convert midi information to numpy array (pianoroll)
pianoroll = np.zeros((88, int(max_end*100)+1))
for instru in data.instruments:
    notes = instru.notes
    for note in notes:

        note_start_time = int(note.start * 100)
        note_end_time = int(note.end * 100) + 1
        note_pitch = note.pitch

        pianoroll[note_pitch][note_start_time:note_end_time] = 1
pianoroll = np.transpose(pianoroll)

In [10]:
# merge row in pianoroll (25 rows => 1 point in time)
total_row = None
if pianoroll.shape[0] % 25 == 0:
    total_row = pianoroll.shape[0] / 25
else:
    total_row = (pianoroll.shape[0] // 25) + 1

In [11]:
pianoroll_processed = np.zeros((total_row, 88))
for i in range(total_row):
    start_idx = 25 * i
    end_idx = min(25*(i+1), pianoroll.shape[0])
    pianoroll_part = pianoroll[start_idx:end_idx][:]
    pianoroll_processed[i][:] = np.any(pianoroll_part, axis=0)

print(f'merged pianoroll array shape: {pianoroll_processed.shape}')

merged pianoroll array shape: (397, 88)


In [12]:
# convert merged pianoroll array to batch of smaples
total_sample = None
if pianoroll_processed.shape[0] % 128 == 0:
    total_sample = pianoroll_processed.shape[0] / 128
else:
    total_sample = (pianoroll_processed.shape[0] // 128) + 1

input_tensor = np.zeros((total_sample, 128, 88))
len_tensor = np.zeros((total_sample))

In [13]:
for i in range(total_sample):
    start_idx = 128 * i
    end_idx = 128*(i+1)
    if end_idx <= pianoroll_processed.shape[0]:
        pianoroll_part = pianoroll_processed[start_idx:end_idx][:]
        input_tensor[i][:][:] = pianoroll_part[:][:]
        len_tensor[i] = pianoroll_part.shape[0]
    else:
        pianoroll_part = pianoroll_processed[start_idx:pianoroll_processed.shape[0]][:]
        input_tensor[i][:pianoroll_part.shape[0]][:] = pianoroll_part[:][:]
        len_tensor[i] = pianoroll_part.shape[0]

input_tensor = input_tensor.astype('int32')
len_tensor = len_tensor.astype('int32')

print(f'input_tensor shape: {input_tensor.shape}')
print(f'len_tensor shape: {len_tensor.shape}')

input_tensor shape: (4, 128, 88)
len_tensor shape: (4,)


In [14]:
np.save("input.npy", input_tensor)
np.save("len.npy", len_tensor)

### Get Pretrained Model

In [15]:
# unzip
path="/gdrive/MyDrive/Classical_Music_Transformer_Integration/data/pretrained_model/chord_recognition/pretrained_model.rar"
!unrar x $path
!rm -r ./sample_data/


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from /gdrive/MyDrive/Classical_Music_Transformer_Integration/data/pretrained_model/chord_recognition/pretrained_model.rar

Extracting  functional_harmony_recognition.py                              0%  OK 
Creating    model_functional                                          OK
Extracting  model_functional/checkpoint                                    0%  OK 
Extracting  model_functional/HT_functional_harmony_recognition_BPS_FH_1.ckpt.data-00000-of-00001       0%  1%  2%  3%  4%  5%  6%  7%  8%  9% 10% 11% 12% 13% 14% 15% 16% 17% 18% 19% 20% 21% 22%  OK 
Extracting  model_functional/HT_functional_harmony_recognition_BPS_FH_1.ckpt.index      22%  OK 
Extracting  model_functional/HT_functional_harmony_recognition_BPS_FH_1.ckpt.meta      22% 23%  OK 
Creating    model_functio

### Model Helper Function

In [16]:
import chord_recognition_models as crm

In [17]:
# Mappings of functional harmony
'''key: 7 degrees * 3 accidentals * 2 modes + 1 padding= 43'''
key_dict = {}
for i_a, accidental in enumerate(['', '#', 'b']):
    for i_t, tonic in enumerate(['C', 'D', 'E', 'F', 'G', 'A', 'B', 'c', 'd', 'e', 'f', 'g', 'a', 'b']):
        key_dict[tonic + accidental] = i_a * 14 + i_t
        if accidental == '#':
            key_dict[tonic + '+'] = i_a * 14 + i_t
        elif accidental == 'b':
            key_dict[tonic + '-'] = i_a * 14 + i_t
key_dict['pad'] = 42

'''degree1: 10 (['1', '2', '3', '4', '5', '6', '7', '-2', '-7', 'pad'])'''
degree1_dict = {d1: i for i, d1 in enumerate(['1', '2', '3', '4', '5', '6', '7', '-2', '-7', 'pad'])}

'''degree2: 15 ['1', '2', '3', '4', '5', '6', '7', '+1', '+3', '+4', '-2', '-3', '-6', '-7', 'pad'])'''
degree2_dict = {d2: i for i, d2 in enumerate(['1', '2', '3', '4', '5', '6', '7', '+1', '+3', '+4', '-2', '-3', '-6', '-7', 'pad'])}

'''quality: 11 (['M', 'm', 'a', 'd', 'M7', 'm7', 'D7', 'd7', 'h7', 'a6', 'pad'])'''
quality_dict = {q: i for i, q in enumerate(['M', 'm', 'a', 'd', 'M7', 'm7', 'D7', 'd7', 'h7', 'a6', 'pad'])}
quality_dict['a7'] = [v for k, v in quality_dict.items() if k == 'a'][0]

In [18]:
def inverse_roman(roman):
    d1 = int(roman / (14 * 10 * 4))
    d2 = int((roman - d1 * 14 * 10 * 4) / (10 * 4))
    q = int((roman - d1 * 14 * 10 * 4 - d2 * 10 * 4) / 4)
    inv = roman - d1 * 14 * 10 * 4 - d2 * 10 * 4 - q * 4
    return (d1, d2, q, inv)

def load_data_functional(dir, test_set_id=1, sequence_with_overlap=True):
    if test_set_id not in [1, 2, 3, 4]:
        print('Invalid testing_set_id.')
        exit(1)

    print("Load functional harmony data ...")
    print('test_set_id =', test_set_id)
    with open(dir, 'rb') as file:
        corpus_aug_reshape = pickle.load(file)
    print('keys in corpus_aug_reshape[\'shift_id\'][\'op\'] =', corpus_aug_reshape['shift_0']['1'].keys())

    shift_list = sorted(corpus_aug_reshape.keys())
    number_of_pieces = len(corpus_aug_reshape['shift_0'].keys())

    # train_op_list = [str(i + 1) for i in range(number_of_pieces) if i % 4 + 1 != test_set_id]
    test_op_list = [str(i + 1) for i in range(number_of_pieces) if i % 4 + 1 == test_set_id]
    
    print('shift_list =', shift_list)
    # print('train_op_list =', train_op_list)
    print('test_op_list =', test_op_list)

    overlap = int(sequence_with_overlap)

    # Training set
    '''
    train_data = {'pianoroll': np.concatenate([corpus_aug_reshape[shift_id][op]['pianoroll'][overlap] for shift_id in shift_list for op in train_op_list], axis=0),
                  'tonal_centroid': np.concatenate([corpus_aug_reshape[shift_id][op]['tonal_centroid'][overlap] for shift_id in shift_list for op in train_op_list], axis=0),
                  'len': np.concatenate([corpus_aug_reshape[shift_id][op]['len'][overlap] for shift_id in shift_list for op in train_op_list], axis=0),
                  'label': np.concatenate([corpus_aug_reshape[shift_id][op]['label'][overlap] for shift_id in shift_list for op in train_op_list], axis=0)}

    train_data_label_key = np.zeros_like(train_data['label'], dtype=np.int32)
    train_data_label_degree1 = np.zeros_like(train_data['label'], dtype=np.int32)
    train_data_label_degree2 = np.zeros_like(train_data['label'], dtype=np.int32)
    train_data_label_quality = np.zeros_like(train_data['label'], dtype=np.int32)
    train_data_label_inversion = train_data['label']['inversion']

    # Functional harmony labels
    # key: 42
    for k, v in key_dict.items():
        train_data_label_key[train_data['label']['key'] == k] = v
    # degree1: 9
    for k, v in degree1_dict.items():
        train_data_label_degree1[train_data['label']['degree1'] == k] = v
    # degree2: 14
    for k, v in degree2_dict.items():
        train_data_label_degree2[train_data['label']['degree2'] == k] = v
    # quality: 10
    for k, v in quality_dict.items():
        train_data_label_quality[train_data['label']['quality'] == k] = v
    # inversion: 4
    train_data_label_inversion[train_data_label_inversion == -1] = 4
    # roman numeral: (degree1, degree2, quality, inversion)
    train_data_label_roman = train_data_label_degree1 * 14 * 10 * 4 + train_data_label_degree2 * 10 * 4 + train_data_label_quality * 4 + train_data_label_inversion
    train_data_label_roman[train_data['label']['key'] == 'pad'] = 9 * 14 * 10 * 4

    train_data['key'] = train_data_label_key
    train_data['degree1'] = train_data_label_degree1
    train_data['degree2'] = train_data_label_degree2
    train_data['quality'] = train_data_label_quality
    train_data['inversion'] = train_data_label_inversion
    train_data['roman'] = train_data_label_roman
    '''
    
    # Test set
    test_data = {'pianoroll': np.concatenate([corpus_aug_reshape['shift_0'][op]['pianoroll'][0] for op in test_op_list], axis=0),
                 'tonal_centroid': np.concatenate([corpus_aug_reshape['shift_0'][op]['tonal_centroid'][0] for op in test_op_list], axis=0),
                 'len': np.concatenate([corpus_aug_reshape['shift_0'][op]['len'][0] for op in test_op_list], axis=0),
                 'label': np.concatenate([corpus_aug_reshape['shift_0'][op]['label'][0] for op in test_op_list], axis=0)}

    test_data_label_key = np.zeros_like(test_data['label'], dtype=np.int32)
    test_data_label_degree1 = np.zeros_like(test_data['label'], dtype=np.int32)
    test_data_label_degree2 = np.zeros_like(test_data['label'], dtype=np.int32)
    test_data_label_quality = np.zeros_like(test_data['label'], dtype=np.int32)
    test_data_label_inversion = test_data['label']['inversion']

    # Functional harmony labels
    '''key: 42'''
    for k, v in key_dict.items():
        test_data_label_key[test_data['label']['key'] == k] = v
    '''degree1: 9'''
    for k, v in degree1_dict.items():
        test_data_label_degree1[test_data['label']['degree1'] == k] = v
    '''degree2: 14'''
    for k, v in degree2_dict.items():
        test_data_label_degree2[test_data['label']['degree2'] == k] = v
    '''quality: 10'''
    for k, v in quality_dict.items():
        test_data_label_quality[test_data['label']['quality'] == k] = v
    '''inversion: 4'''
    test_data_label_inversion[test_data_label_inversion == -1] = 4
    '''roman numeral'''
    test_data_label_roman = test_data_label_degree1 * 14 * 10 * 4 + test_data_label_degree2 * 10 * 4 + test_data_label_quality * 4 + test_data_label_inversion
    test_data_label_roman[test_data['label']['key'] == 'pad'] = 9 * 14 * 10 * 4

    test_data['key'] = test_data_label_key
    test_data['degree1'] = test_data_label_degree1
    test_data['degree2'] = test_data_label_degree2
    test_data['quality'] = test_data_label_quality
    test_data['inversion'] = test_data_label_inversion
    test_data['roman'] = test_data_label_roman

    # print('keys in train/test_data =', train_data.keys())
    # return train_data, test_data
    return test_data

def compute_pre_PRF(predicted, actual):
    predicted = tf.cast(predicted, tf.float32)
    actual = tf.cast(actual, tf.float32)
    TP = tf.count_nonzero(predicted * actual, dtype=tf.float32)
    # TN = tf.count_nonzero((predicted - 1) * (actual - 1), dtype=tf.float32)
    FP = tf.count_nonzero(predicted * (actual - 1), dtype=tf.float32)
    FN = tf.count_nonzero((predicted - 1) * actual, dtype=tf.float32)
    return TP, FP, FN

def comput_PRF_with_pre(TP, FP, FN):
    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    F1 = 2 * precision * recall / (precision + recall)
    precision = tf.cond(tf.is_nan(precision), lambda: tf.constant(0.0), lambda: precision)
    recall = tf.cond(tf.is_nan(recall), lambda: tf.constant(0.0), lambda: recall)
    F1 = tf.cond(tf.is_nan(F1), lambda: tf.constant(0.0), lambda: F1)
    return precision, recall, F1

### Model Setting

In [19]:
hyperparameters = namedtuple('hyperparameters',
                                 ['dataset',
                                  'test_set_id',
                                  'graph_location',
                                  'n_steps',
                                  'input_embed_size',
                                  'n_layers',
                                  'n_heads',
                                  'train_sequence_with_overlap',
                                  'initial_learning_rate',
                                  'drop',
                                  'n_batches',
                                  'n_training_steps',
                                  'n_in_succession',
                                  'annealing_rate'])

hp = hyperparameters(dataset='BPS_FH', # {'BPS_FH', 'Preludes'}
                        test_set_id=1,
                        graph_location='model',
                        n_steps=128,
                        input_embed_size=128,
                        n_layers=2,
                        n_heads=4,
                        train_sequence_with_overlap=True,
                        initial_learning_rate=1e-4,
                        drop=0.1,
                        n_batches=40,
                        n_training_steps=100000,
                        n_in_succession=10,
                        annealing_rate=1.1)

In [20]:
test_data = load_data_functional(dir=hp.dataset + '_preprocessed_data_MIREX_Mm.pickle', test_set_id=hp.test_set_id, sequence_with_overlap=hp.train_sequence_with_overlap)

Load functional harmony data ...
test_set_id = 1
keys in corpus_aug_reshape['shift_id']['op'] = dict_keys(['pianoroll', 'tonal_centroid', 'start_time', 'label', 'len'])
shift_list = ['shift_-1', 'shift_-2', 'shift_-3', 'shift_0', 'shift_1', 'shift_2', 'shift_3', 'shift_4', 'shift_5', 'shift_6']
test_op_list = ['1', '5', '9', '13', '17', '21', '25', '29']


In [21]:
# load npy
input_tensor = np.load('input.npy')
len_tensor = np.load('len.npy')

test_data['pianoroll'] = input_tensor
test_data['len'] = len_tensor

In [22]:
# Load training and testing data
n_test_sequences = test_data['pianoroll'].shape[0]
print('n_test_sequences =', n_test_sequences)

n_test_sequences = 4


In [23]:
with tf.name_scope('placeholder'):
    x_p = tf.placeholder(tf.int32, [None, hp.n_steps, 88], name="pianoroll")
    x_len = tf.placeholder(tf.int32, [None], name="seq_lens")
    # y_k = tf.placeholder(tf.int32, [None, hp.n_steps], name="key") # 7 degrees * 3 accidentals * 2 modes = 42
    # y_r = tf.placeholder(tf.int32, [None, hp.n_steps], name="roman_numeral")
    # y_cc = tf.placeholder(tf.int32, [None, hp.n_steps], name="chord_change")
    dropout = tf.placeholder(dtype=tf.float32, name="dropout_rate")
    is_training = tf.placeholder(dtype=tf.bool, name="is_training")
    # global_step = tf.placeholder(dtype=tf.int32, name='global_step')
    slope = tf.placeholder(dtype=tf.float32, name='annealing_slope')

In [24]:
with tf.name_scope('model'):
    x_in = tf.cast(x_p, tf.float32)
    source_mask = tf.sequence_mask(lengths=x_len, maxlen=hp.n_steps, dtype=tf.float32) # [n_batches, n_steps]
    target_mask = source_mask
    chord_change_logits, dec_input_embed, enc_weights, dec_weights, _, _ = crm.HTv2(x_in, source_mask, target_mask, slope, dropout, is_training, hp)


Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.dropout instead.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use `tf.keras.layers.Conv1D` instead.




In [25]:
with tf.variable_scope("output_projection"):
    n_key_classes = 42 + 1
    n_roman_classes = 9 * 14 * 10 * 4 + 1
    dec_input_embed = tf.layers.dropout(dec_input_embed, rate=dropout, training=is_training)
    key_logits = tf.layers.dense(dec_input_embed, n_key_classes)
    roman_logits = tf.layers.dense(dec_input_embed, n_roman_classes)

In [26]:
with tf.name_scope('inference'):
    eval_mask = tf.cast(target_mask, tf.bool)
    # Chord change
    pred_cc = tf.cast(tf.round(tf.sigmoid(slope*chord_change_logits)), tf.int32)
    # pred_cc_mask = tf.boolean_mask(pred_cc, tf.cast(source_mask, tf.bool))
    # y_cc_mask = tf.boolean_mask(y_cc, tf.cast(source_mask, tf.bool))
    # TP_cc, FP_cc, FN_cc = compute_pre_PRF(pred_cc_mask, y_cc_mask)
    # Key
    pred_k = tf.argmax(key_logits, axis=2, output_type=tf.int32)
    # pred_k_correct = tf.equal(pred_k, y_k)
    # pred_k_correct_mask = tf.boolean_mask(tensor=pred_k_correct, mask=eval_mask)
    # n_correct_k = tf.reduce_sum(tf.cast(pred_k_correct_mask, tf.float32))
    # Roman numeral
    pred_r = tf.argmax(roman_logits, axis=2, output_type=tf.int32)
    # pred_r_correct = tf.equal(pred_r, y_r)
    # pred_r_correct_mask = tf.boolean_mask(tensor=pred_r_correct, mask=eval_mask)
    # n_correct_r = tf.reduce_sum(tf.cast(pred_r_correct_mask, tf.float32))
    # n_total = tf.cast(tf.size(pred_r_correct_mask), tf.float32)

In [27]:
saver = tf.train.Saver(max_to_keep=1)

### Model Inference

In [28]:
with tf.Session() as sess:
    annealing_slope = 3.797498335832415
    saver.restore(sess, save_path='./model_functional/HT_functional_harmony_recognition_BPS_FH_1.ckpt')

    test_run_list = [pred_cc, pred_k, pred_r, eval_mask]
    test_feed_fict = {x_p: test_data['pianoroll'],
                        x_len: test_data['len'],
                        dropout: 0.0,
                        is_training: False,
                        slope: annealing_slope}
    test_pred_cc, test_pred_k, test_pred_r, test_eval_mask = sess.run(test_run_list, feed_dict=test_feed_fict)

INFO:tensorflow:Restoring parameters from ./model_functional/HT_functional_harmony_recognition_BPS_FH_1.ckpt


### Parse Model Inference Result

In [29]:
inverse_key_dict = {v: k for k, v in key_dict.items()}

In [30]:
cc = []
k = []
d1 = []
d2 = []
q = []
inv = []

for sample_id in range(test_data['len'].shape[0]):

    valid = test_eval_mask[sample_id]
    pred_cc = test_pred_cc[sample_id]
    pred_k = test_pred_k[sample_id]
    pred_r = test_pred_r[sample_id]

    for idx in range(len(valid)):

        if valid[idx]:

            # k
            key = pred_k[idx]
            key = inverse_key_dict[key]
            #if key == 'pad':
            #    continue
            k.append(key)

            # cc
            cc.append(str(pred_cc[idx]))

            # d1
            roman = pred_r[idx]
            d1.append(str(inverse_roman(roman)[0]))

            # d2
            d2.append(str(inverse_roman(roman)[1]))

            # q
            q.append(str(inverse_roman(roman)[2]))

            # inv
            inv.append(str(inverse_roman(roman)[3]))

In [31]:
df_dict = {
    'cc': cc,
    'k': k,
    'd1': d1,
    'd2': d2,
    'q': q,
    'inv': inv
}

df = pd.DataFrame.from_dict(df_dict)

In [32]:
df

,cc,k,d1,d2,q,inv
0,0,E,0,0,0,0
1,0,E,0,0,0,0
2,0,E,0,0,0,0
3,0,E,0,0,0,0
4,0,E,0,0,0,0
...,...,...,...,...,...,...
392,0,a,0,0,1,0
393,0,a,0,0,1,0
394,0,a,0,0,1,0
395,0,a,0,0,1,0


In [33]:
# d1 table
d1_table = {
    '0': '1',
    '1': '2',
    '2': '3',
    '3': '4',
    '4': '5',
    '5': '6',
    '6': '7',
    '7': '-2',
    '8': '-7',
    '9': 'pad',
}

# d2 table
d2_table = {
    '0': '1',
    '1': '2',
    '2': '3',
    '3': '4',
    '4': '5',
    '5': '6',
    '6': '7',
    '7': '+1',
    '8': '+3',
    '9': '+4',
    '10': '-2',
    '11': '-3',
    '12': '-6',
    '13': '-7',
    '14': 'pad',
}


# q table
q_table = {
    '0': 'M',
    '1': 'm',
    '2': 'a',
    '3': 'd',
    '4': 'M7',
    '5': 'm7',
    '6': 'D7',
    '7': 'd7',
    '8': 'h7',
    '9': 'a6',
    '10': 'pad',
}

def transform_d1(key):
    return d1_table[key]

def transform_d2(key):
    return d2_table[key]

def transform_q(key):
    return q_table[key]

In [34]:
df['d1'] = df['d1'].apply(transform_d1)
df['d2'] = df['d2'].apply(transform_d2)
df['q'] = df['q'].apply(transform_q)

In [35]:
start_arr = np.array(df.index) * 0.25
end_arr = start_arr[1:]
end_arr = np.append(end_arr, end_arr[-1]+0.25)

df['start'] = start_arr
df['end'] = end_arr

In [36]:
def concat_d1_d2(value):

    d1 = value[0]
    d2 = value[1]
    
    if d1 == '1':
        return d2
    else:
        return d2 + '/' + d1
    

df['d2/d1'] = df[['d1', 'd2']].apply(concat_d1_d2, axis=1)

In [37]:
df

,cc,k,d1,d2,q,inv,start,end,d2/d1
0,0,E,1,1,M,0,0.00,0.25,1
1,0,E,1,1,M,0,0.25,0.50,1
2,0,E,1,1,M,0,0.50,0.75,1
3,0,E,1,1,M,0,0.75,1.00,1
4,0,E,1,1,M,0,1.00,1.25,1
...,...,...,...,...,...,...,...,...,...
392,0,a,1,1,m,0,98.00,98.25,1
393,0,a,1,1,m,0,98.25,98.50,1
394,0,a,1,1,m,0,98.50,98.75,1
395,0,a,1,1,m,0,98.75,99.00,1


In [38]:
def d2_num2roman(num):

    if num == '1' or num == '+1':
        return 'i'
    elif num == '2' or num == '+3':
        return 'ii'
    elif num == '3' or num == '+4':
        return 'iii'
    elif num == '4' or num == '-2':
        return 'iv'
    elif num == '5' or num == '-3':
        return 'v'
    elif num == '6' or num == '-6':
        return 'vi'
    elif num == '7' or num == '-7':
        return 'vii'
    else:
        return 'ERROR'

def d1_num2roman(num):

    if num == '1' or num == '-2':
        return 'i'
    elif num == '2' or num == '-7':
        return 'ii'
    elif num == '3':
        return 'iii'
    elif num == '4':
        return 'iv'
    elif num == '5':
        return 'v'
    elif num == '6':
        return 'vi'
    elif num == '7':
        return 'vii'
    else:
        return 'ERROR'


def generate_roman(value):

    d1d2 = value[0]
    quality = value[1]

    if '/' in d1d2:
        d2 = d1d2.split('/')[0]
        d1 = d1d2.split('/')[1]

        roman2 = d2_num2roman(d2)
        roman1 = d1_num2roman(d1)
        
        if quality[0] != 'm':
            roman2 = roman2.upper()
            roman1 = roman1.upper()
            
        if len(quality) == 2:
            roman2 += quality[-1]

        return f'{roman2}/{roman1}'
    else:
        d2 = d1d2
        roman2 = d2_num2roman(d2)

        if quality[0] != 'm':
            roman2 = roman2.upper()
        
        if len(quality) == 2:
            roman2 += quality[-1]
        
        return roman2

In [39]:
df['roman'] = df[['d2/d1', 'q']].apply(generate_roman, axis=1)

In [40]:
df

,cc,k,d1,d2,q,inv,start,end,d2/d1,roman
0,0,E,1,1,M,0,0.00,0.25,1,I
1,0,E,1,1,M,0,0.25,0.50,1,I
2,0,E,1,1,M,0,0.50,0.75,1,I
3,0,E,1,1,M,0,0.75,1.00,1,I
4,0,E,1,1,M,0,1.00,1.25,1,I
...,...,...,...,...,...,...,...,...,...,...
392,0,a,1,1,m,0,98.00,98.25,1,i
393,0,a,1,1,m,0,98.25,98.50,1,i
394,0,a,1,1,m,0,98.50,98.75,1,i
395,0,a,1,1,m,0,98.75,99.00,1,i


In [41]:
df[['start', 'end', 'k', 'd2/d1', 'q', 'inv', 'roman']].to_excel('chords.xlsx', index=False, header=False)

### Debug

In [42]:
# sample_id = 9

In [43]:
# print('sample_id = ', sample_id)
# print('len =', test_data['len'][sample_id])

In [44]:
# print('valid: ', ''.join([' y' if b else ' n' for b in test_eval_mask[sample_id]]))

In [45]:
# print('pred_cc: ', ''.join([' '+str(b) for b in test_pred_cc[sample_id]]))

In [46]:
# print('pred_k: ', ''.join([' '+inverse_key_dict[key] for key in test_pred_k[sample_id]]))

In [47]:
# print('pred_d1: ', ''.join([' '+str(inverse_roman(roman)[0]) for roman in test_pred_r[sample_id]]))

In [48]:
# print('pred_d2: ', ''.join([' '+str(inverse_roman(roman)[1]) for roman in test_pred_r[sample_id]]))

In [49]:
# print('pred_q: ', ''.join([' '+str(inverse_roman(roman)[2]) for roman in test_pred_r[sample_id]]))

In [50]:
# print('pred_inv: ', ''.join([' '+str(inverse_roman(roman)[3]) for roman in test_pred_r[sample_id]]))

## Extract Note and Beat

### Extract Note

In [51]:
# read midi file
data = pretty_midi.PrettyMIDI(midi_src_path)
print(f'num of instruments: {len(data.instruments)}')

num of instruments: 3


In [52]:
all_note = []

for instru in data.instruments[:]:
    instru_notes = instru.notes

    for note in instru_notes:
        note_lst = []

        # onset
        note_lst.append(note.start)

        # pitch
        note_lst.append(note.pitch)

        # mpitch
        note_lst.append(-1)

        # duration
        note_lst.append(note.end - note.start)

        # staffnum
        note_lst.append(-1)

        # measure
        note_lst.append(-1)

        all_note.append(note_lst)

In [53]:
all_note = sorted(all_note, key=lambda note:note[0])
all_note = np.array(all_note)

In [54]:
np.savetxt('notes.csv', np.array(all_note), delimiter=',', fmt=['%f', '%d', '%d', '%f', '%d', '%d']) 

### Extract Beat

In [55]:
values = data.get_beats()
beat_dict = {
    'beat': values
}

In [56]:
pd.DataFrame(beat_dict).to_excel('beats.xlsx')

### Extract Down Beat

In [57]:
values = data.get_downbeats()
d_beat_dict = {
    'd_beat': values
}

In [58]:
pd.DataFrame(d_beat_dict).to_excel('dBeats.xlsx')

## Collect Note, Beat and Chord

In [59]:
name = midi_src_path.split('/')[-1].split('.')[0]
path = "/gdrive/MyDrive/Classical_Music_Transformer_Integration/data/preprocessed_data/" + name
!mkdir $path

In [60]:
!mv -v notes.csv $path
!mv -v beats.xlsx $path
!mv -v dBeats.xlsx $path
!mv -v chords.xlsx $path

copied 'notes.csv' -> '/gdrive/MyDrive/Classical_Music_Transformer_Integration/data/preprocessed_data/笑忘歌_120/notes.csv'
removed 'notes.csv'
copied 'beats.xlsx' -> '/gdrive/MyDrive/Classical_Music_Transformer_Integration/data/preprocessed_data/笑忘歌_120/beats.xlsx'
removed 'beats.xlsx'
copied 'dBeats.xlsx' -> '/gdrive/MyDrive/Classical_Music_Transformer_Integration/data/preprocessed_data/笑忘歌_120/dBeats.xlsx'
removed 'dBeats.xlsx'
copied 'chords.xlsx' -> '/gdrive/MyDrive/Classical_Music_Transformer_Integration/data/preprocessed_data/笑忘歌_120/chords.xlsx'
removed 'chords.xlsx'
